In [ ]:
# creatting library management system demo

In [1]:
import psycopg2


In [32]:
conn = psycopg2.connect(
    host="localhost",
    port="5432",
    database="Library_Demo",
    user="postgres",
    password="password"
)


In [33]:
# Create a cursor object
cursor = conn.cursor()

In [44]:
# Execute the SQL scripts to create tables
create_table_queries = [
    """
    CREATE TABLE books (
      book_id serial PRIMARY KEY,
      title varchar(1000),
      isbn bigint NOT NULL UNIQUE,
      publication_year int,
      publisher varchar(200),
      nr_of_pages int,
      summary varchar(2000),
      category_id smallint
    )
    """,
    """
    CREATE TABLE users (
      user_id serial PRIMARY KEY,
      first_name varchar(200),
      last_name varchar(200),
      phone_nr varchar(20),
      membership_expiry date,
      join_date date,
      email varchar(200)
    )
    """,
    """
    CREATE TABLE authors (
      author_id serial PRIMARY KEY,
      first_name varchar(200),
      last_name varchar(200)
    )
    """,
    """
    CREATE TABLE copies (
      copy_id serial PRIMARY KEY,
      book_id int REFERENCES books(book_id),
      available boolean
    )
    """,
    """
    CREATE TABLE loans (
      loan_id serial PRIMARY KEY,
      user_id int REFERENCES users(user_id),
      copy_id int REFERENCES copies(copy_id),
      loan_start_date date,
      loan_end_date date,
      return_date date
    )
    """,
    """
    CREATE TABLE fines (
      fine_id serial PRIMARY KEY,
      loan_id int REFERENCES loans(loan_id),
      fine_amount int,
      issue_date date,
      payment_date date,
      paid boolean
    )
    """,
    """
    CREATE TABLE book_reviews (
      review_id serial PRIMARY KEY,
      book_id int REFERENCES books(book_id),
      user_id int REFERENCES users(user_id),
      rating smallint,
      review varchar,
      review_date date
    )
    """,
    """
    CREATE TABLE book_author_link (
      book_id int REFERENCES books(book_id),
      author_id int REFERENCES authors(author_id),
      PRIMARY KEY (book_id, author_id)
    )
    """
]

for query in create_table_queries:
    cursor.execute(query)

# Commit the changes
conn.commit()


In [47]:
# Execute the SQL scripts to insert example data
insert_data_queries = [
        """
        INSERT INTO books (title, isbn, publication_year, publisher, nr_of_pages, summary, category_id)
        VALUES 
            ('To Kill a Mockingbird', 9780061120084, 1960, 'HarperCollins', 281, 'To Kill a Mockingbird is a novel by Harper Lee.', 1),
            ('Pride and Prejudice', 9780141439518, 1813, 'Penguin Classics', 432, 'Pride and Prejudice is a romantic novel by Jane Austen.', 2),
            ('The Great Gatsby', 9780743273565, 1925, 'Scribner', 180, 'The Great Gatsby is a novel by F. Scott Fitzgerald.', 1);
        """,
        """
        INSERT INTO users (first_name, last_name, phone_nr, membership_expiry, join_date, email)
        VALUES 
            ('John', 'Doe', '123456789', '2023-12-31', '2020-01-01', 'john.doe@example.com'),
            ('Jane', 'Smith', '987654321', '2024-06-30', '2021-03-15', 'jane.smith@example.com'),
            ('Mike', 'Johnson', '555555555', '2023-10-15', '2019-07-01', 'mike.johnson@example.com');
        """,
        """
        INSERT INTO authors (first_name, last_name)
        VALUES 
            ('Harper', 'Lee'),
            ('Jane', 'Austen'),
            ('F. Scott', 'Fitzgerald');
        """,
        """
        INSERT INTO copies (book_id, available)
        VALUES 
            (1, true),
            (2, true),
            (3, true);
        """,
        """
        INSERT INTO loans (user_id, copy_id, loan_start_date, loan_end_date, return_date)
        VALUES 
            (1, 1, '2021-02-01', '2021-02-15', '2021-02-14'),
            (2, 2, '2022-04-01', '2022-04-15', NULL),
            (3, 3, '2023-01-01', '2023-01-15', NULL);
        """,
        """
        INSERT INTO fines (loan_id, fine_amount, issue_date, payment_date, paid)
        VALUES 
            (1, 10, '2021-02-16', NULL, false),
            (2, 5, '2022-04-16', NULL, false),
            (3, 15, '2023-01-16', NULL, false);
        """,
        """
        INSERT INTO book_reviews (book_id, user_id, rating, review, review_date)
        VALUES 
            (1, 1, 4, 'Highly recommended!', '2021-02-15'),
            (2, 2, 3, 'Enjoyable read.', '2022-04-15'),
            (3, 3, 5, 'One of my favorites!', '2023-01-15');
        """,
        """
        INSERT INTO book_author_link (book_id, author_id)
        VALUES 
            (1, 1),
            (1, 3),
            (2, 2);
        """
    ]

for query in insert_data_queries:
    try:
        cursor.execute(query)
        conn.commit()
    except psycopg2.Error as e:
        conn.rollback()
        print("Error executing query:", query)
        print("Exception:", e)
    
    

In [48]:
conn.close()
